In [1]:
import pandas as pd
import warnings
import numpy as np
import sys
import os
import yfinance as yf

In [2]:
sys.path.append(os.path.abspath('C:/Users/teeyob/Predictive_Portfolio_Optimizer/scripts'))

In [3]:
from time_series_forecasting import train_arima, train_sarima, train_lstm, forecast_lstm, evaluate_forecast,forecast_arima
from data_analysis_functions import download_data,clean_data

In [4]:
tickers = ['TSLA', 'BND', 'SPY']
start_date = '2015-01-01'
end_date = '2024-10-31'

data = yf.download(tickers, start=start_date, end=end_date)
    

[*********************100%***********************]  3 of 3 completed


In [9]:
data = clean_data(data)
data.head

<bound method NDFrame.head of Price                      Adj Close                              Close  \
Ticker                           BND         SPY        TSLA        BND   
Date                                                                      
2015-01-02 00:00:00+00:00  63.358742  173.173752   14.620667  82.650002   
2015-01-05 00:00:00+00:00  63.542725  170.046341   14.006000  82.889999   
2015-01-06 00:00:00+00:00  63.726727  168.444626   14.085333  83.129997   
2015-01-07 00:00:00+00:00  63.765099  170.543640   14.063333  83.180000   
2015-01-08 00:00:00+00:00  63.665394  173.569992   14.041333  83.050003   
...                              ...         ...         ...        ...   
2024-10-24 00:00:00+00:00  73.081184  579.239990  260.480011  73.309998   
2024-10-25 00:00:00+00:00  72.931664  579.039978  269.190002  73.160004   
2024-10-28 00:00:00+00:00  72.841942  580.830017  262.510010  73.070000   
2024-10-29 00:00:00+00:00  72.901749  581.770020  259.519989  73.12999

In [ ]:
train_data = data[('Adj Close', 'TSLA')][:'2022']
test_data = data[('Adj Close', 'TSLA')]['2023':]

Ensure the train data index is a DatetimeIndex

In [15]:
train_data.index = pd.to_datetime(train_data.index)
test_data.index = pd.to_datetime(test_data.index)
warnings.filterwarnings("ignore", message="No supported index is available")

Train ARIMA model

In [16]:
arima_model = train_arima(train_data, order=(1, 1, 1))

c:\Users\teeyob\Predictive_Portfolio_Optimizer\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\teeyob\Predictive_Portfolio_Optimizer\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\teeyob\Predictive_Portfolio_Optimizer\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Forecast using the trained ARIMA model

In [ ]:
arima_forecast = arima_model.get_forecast(steps=len(test_data)).predicted_mean
arima_forecast.index = test_data.index  

print("ARIMA Evaluation:", evaluate_forecast(test_data, arima_forecast))

print("First few rows of the forecast:")
print(arima_forecast.head())

ARIMA Evaluation: {'MAE': np.float64(88.21168239768238), 'RMSE': 95.62463833667206, 'MAPE': np.float64(39.81058208390919)}
First few rows of the forecast:
Date
2023-01-03 00:00:00+00:00    123.201250
2023-01-04 00:00:00+00:00    123.190937
2023-01-05 00:00:00+00:00    123.195942
2023-01-06 00:00:00+00:00    123.193513
2023-01-09 00:00:00+00:00    123.194692
Name: predicted_mean, dtype: float64


SARIMA Model

In [ ]:
sarima_model = train_sarima(train_data, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
sarima_predictions = sarima_model.forecast(steps=len(test_data))
print("SARIMA Evaluation:", evaluate_forecast(test_data, sarima_predictions))

c:\Users\teeyob\Predictive_Portfolio_Optimizer\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\teeyob\Predictive_Portfolio_Optimizer\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


SARIMA Evaluation: {'MAE': np.float64(77.28267438335756), 'RMSE': 85.49586439089785, 'MAPE': np.float64(34.59939277014497)}


LSTM Model

In [19]:
lstm_model, n_steps = train_lstm(train_data.values, n_steps=50)
lstm_predictions = forecast_lstm(lstm_model, test_data.values, n_steps=n_steps)
print("LSTM Evaluation:", evaluate_forecast(test_data.values[n_steps:], lstm_predictions))

c:\Users\teeyob\Predictive_Portfolio_Optimizer\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 23920.4902
Epoch 2/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 5341.0903
Epoch 3/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 1388787.6250
Epoch 4/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 16015830.0000
Epoch 5/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 2423.0764
Epoch 6/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 1300.1147
Epoch 7/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 4400.1118
Epoch 8/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 1249.7316
Epoch 9/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 1371.1469
Epoch 10/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 1311.3665
Epoch 11/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 1290.8428
Epoch 12/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 1213.2758
Epoch 13/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 1223.6681
Epoch 14/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 1194.5625
Epoch 15/20
62/62 ━━━━